In [1]:
import pymongo
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
import string
from slimit import ast
from slimit.parser import Parser
from slimit.visitors import nodevisitor

In [2]:
import spacy

In [4]:
# connecting to my mongodb and get data out
client = pymongo.MongoClient('mongodb://localhost:27017/')
# create database
db = client.movie_db
# create colletions
col = db['rotten_tomato']

In [5]:
# get pending movie list fron mongo db
def get_latest_pending_list():
    tomato_data = pd.DataFrame(list(col.find()))
    goal_list = tomato_data.loc[tomato_data['critic(%)'] == 'na'][['name', 'url', 'year']].reset_index()
    
    return goal_list

In [6]:
pending_list = get_latest_pending_list()
pending_list

,index,name,url,year
0,0,Star Wars Ep. VII: The Force Awakens,https://www.rottentomatoes.com/m/star_wars_ep_...,2015
1,6,The Avengers,https://www.rottentomatoes.com/m/the_avengers,2012
2,7,Star Wars Ep. VIII: The Last Jedi,https://www.rottentomatoes.com/m/star_wars_ep_...,2017
3,13,Star Wars Ep. I: The Phantom Menace,https://www.rottentomatoes.com/m/star_wars_ep_...,1999
4,14,Star Wars Ep. IV: A New Hope,https://www.rottentomatoes.com/m/star_wars_ep_...,1977
5,20,Pirates of the Caribbean: Dead Manâs Chest,https://www.rottentomatoes.com/m/pirates_of_th...,2006
6,36,Star Wars Ep. III: Revenge of the Sith,https://www.rottentomatoes.com/m/star_wars_ep_...,2005
7,59,It,https://www.rottentomatoes.com/m/it,2017
8,70,Star Wars Ep. II: Attack of the Clones,https://www.rottentomatoes.com/m/star_wars_ep_...,2002
9,72,Star Wars Ep. VI: Return of the Jedi,https://www.rottentomatoes.com/m/star_wars_ep_...,1983


In [7]:
#search movie in rotten tomato
base_url = 'https://www.rottentomatoes.com/search/?search='
# request url
def get_html(url):
    try:
        r = requests.get(url, timeout=120)
        r.raise_for_status
        #site encoding
        r.encoding = 'utf-8'
        
        return r.text
    except:
        print(url + 'ERROR')
        pass
    
def get_movie_list(url):
    html = get_html(url)
    soup = bs(html, 'lxml')
    movie_list = {}
    try:
        movie_list = soup.find_all('script')[32].text
    except:
        movie_list = 'na'
        
    return movie_list

def get_movie_url(data, movie, year):
    text = data
    parser = Parser()
    tree = parser.parse(text)
    # put the node data into a dict
    fields = []
    url_s = 'to find'
    for node in nodevisitor.visit(tree):
        if isinstance(node, ast.Assign):
            try:
                field = {str(node.left.value).replace('"',''): str(node.right.value).replace('"','')}
                fields.append(field)   
            except:
                pass
    # check title and year
    #print(fields)
    for index, field in enumerate(fields):
        try:
            if(field['name'] == movie):
                year_dic = fields[index + 1]
                url = fields[index + 2]
                
                if year_dic['year'] == str(year):
                    url_s = url['url']
                
        except:
            pass
        
    return url_s

In [58]:
# request url
def get_html(url):
    try:
        r = requests.get(url, timeout=100)
        r.raise_for_status
        #site encoding
        r.encoding = 'utf-8'
        return r.text
    except:
        print('ERROR')
        pass

def get_content(url):
    html = get_html(url)
    soup = bs(html, 'lxml')
    data = {}
    try:
        try:
            data['critic(%)'] = soup.find('span', class_='superPageFontColor').span.text
        except:
            data['critic(%)'] = 'na'
            
        try:
            data['critic_rate'] = soup.find('div', id='scoreStats').findChildren()[0].text[25:].strip()
        except:
            data['critic_rate'] = 'na'
            
        try:
            data['critic_count'] = soup.find('div', id='scoreStats').findChildren()[2].text[18:].strip()
        except:
            data['critic_count'] = 'na'
            
        try:
            data['critic_fresh'] = soup.find('div', id='scoreStats').findChildren()[7].text
        except:
            data['critic_fresh'] = 'na'
        
        try:
            data['critic_rot'] = soup.find('div', id='scoreStats').findChildren()[10].text
        except:
            data['critic_rot'] = 'na'
            
        try:
            data['user(%)'] = soup.find('div', class_='audience-score').find('span', class_='superPageFontColor').text[:2]
        except:
            data['user(%)'] = 'na'
            
        try:
            data['user_rate'] = soup.find('div', class_='audience-info').findChildren()[0].contents[2].strip()
        except:
            data['user_rate'] = 'na'
        
        try:
            data['user_number'] = soup.find('div', class_='audience-info').findChildren()[2].contents[2].strip()
        except:
            data['user_number'] = 'na'

    except:
        print('error')
    
    return data

In [32]:
# check word similarity:
def check_similarity(name1, name2):
    nlp = spacy.load('en')
    check_1 = nlp(name1)
    check_2 = nlp(name2)
    #similarity index
    simi_index = check_1.similarity(check_2)
    print(simi_index)
    return(simi_index)

In [47]:
# blur check function
def similar_check(data, movie, year):
    movie = movie
    year = year
    data = data
    
    parser = Parser()
    tree = parser.parse(data)
    # put the node data into a dict
    fields = []
    url_s = 'to find'
    for node in nodevisitor.visit(tree):
        if isinstance(node, ast.Assign):
            try:
                field = {str(node.left.value).replace('"',''): str(node.right.value).replace('"','')}
                fields.append(field)   
            except:
                pass
    # check title and year
    #print(fields)
    for index, field in enumerate(fields):
        try:
            print(field['name'])
            simi_index = check_similarity(movie, field['name'])
            
            # check if the movie is the one we need
            if(simi_index >= 0.85):
                year_dic = fields[index + 1]
                url = fields[index + 2]
                # check if release year match 
                if year_dic['year'] == str(year):
                    url_s = url['url']
                    print(url_s, simi_index)   
                
        except:
            pass
        
    return url_s

In [74]:
# get url function test
try_url = base_url + 'The Empire Strikes Back'
data = get_movie_list(try_url)
url_find = similar_check(data, 'Star Wars Ep. V: The Empire Strikes Back', '1980')
url_find

Star Wars: Episode V - The Empire Strikes Back
0.8858046971742518
/m/empire_strikes_back 0.8858046971742518
Mark Hamill
0.656010612329752
Harrison Ford
0.6751207995333454
Carrie Fisher
0.6470642812750267


'/m/empire_strikes_back'

In [46]:
key_words = []
for item in pending_list['name']:
    if(":" in item):
        item_list = item.split(':')
        key_words.append({item: item_list})
    else:
        key_words.append({item: [item]})

key_words

[{'Star Wars Ep. VII: The Force Awakens': ['Star Wars Ep. VII',
   ' The Force Awakens']},
 {'The Avengers': ['The Avengers']},
 {'Star Wars Ep. VIII: The Last Jedi': ['Star Wars Ep. VIII',
   ' The Last Jedi']},
 {'Star Wars Ep. I: The Phantom Menace': ['Star Wars Ep. I',
   ' The Phantom Menace']},
 {'Star Wars Ep. IV: A New Hope': ['Star Wars Ep. IV', ' A New Hope']},
 {'Pirates of the Caribbean: Dead Manâ\x80s Chest': ['Pirates of the Caribbean',
   ' Dead Manâ\x80s Chest']},
 {'Star Wars Ep. III: Revenge of the Sith': ['Star Wars Ep. III',
   ' Revenge of the Sith']},
 {'It': ['It']},
 {'Star Wars Ep. II: Attack of the Clones': ['Star Wars Ep. II',
   ' Attack of the Clones']},
 {'Star Wars Ep. VI: Return of the Jedi': ['Star Wars Ep. VI',
   ' Return of the Jedi']},
 {'The Chronicles of Narnia: The Lion, the Witch and the Wardrobe': ['The Chronicles of Narnia',
   ' The Lion, the Witch and the Wardrobe']},
 {'Star Wars Ep. V: The Empire Strikes Back': ['Star Wars Ep. V',
   ' The

In [75]:
find_movie_list = []
for index, row in pending_list.iterrows():
    words = key_words[index][row['name']]
    for word in words:
        url_find_base = 'https://www.rottentomatoes.com'
        url_try = base_url + word
        data = get_movie_list(url_try)
        # similar title check
        url_find = similar_check(data, row['name'], row['year'])
        # put data into a list
        if(url_find != 'to find'):
            export_url = url_find_base + url_find
        else:
            export_url = url_find
        
        find_movie_list.append({'movie': row['name'],
                                'url': export_url,
                                'year': row['year']})
            
        print(row['name'], url_find)

Star Wars Ep. VII: The Force Awakens to find


Power Rangers Wild Force: Ancient Awakening
0.7666664359243
Star Wars: Episode VII - The Force Awakens
0.8647951171179379
/m/star_wars_episode_vii_the_force_awakens 0.8647951171179379
Daisy Ridley
0.5955912283488696
John Boyega
0.541479131039363
Adam Driver
0.6004470405101336
Secrets of The Force Awakens: Chapter 3
0.7235744307176606
J.J. Abrams
0.6508519781346769
Star Wars Ep. VII: The Force Awakens /m/star_wars_episode_vii_the_force_awakens


The Avengers to find


Star Wars Ep. VIII: The Last Jedi to find


Star Wars: The Last Jedi
0.9180620752178079
/m/star_wars_the_last_jedi 0.9180620752178079
Mark Hamill
0.6081440283292169
Carrie Fisher
0.6103057511528239
Adam Driver
0.5204595398982172
Star Wars: The Last Jedi - Opening Night Fan Event
0.8256533338559296
Star Wars Ep. VIII: The Last Jedi /m/star_wars_the_last_jedi


...

Star Wars: Episode VI - Return of the Jedi
0.9003054105582042
/m/star_wars_episode_vi_return_of_the_jedi 0.9003054105582042
Mark Hamill
0.6928583467897903
Carrie Fisher
0.6942503105424478
Harrison Ford
0.7264811485882317
Star Wars Ep. VI: Return of the Jedi /m/star_wars_episode_vi_return_of_the_jedi


The Chronicles of Narnia: The Lion, The Witch and The Wardrobe
0.9875352211799856
/m/chronicles_of_narnia_lion_witch_wardrobe 0.9875352211799856
Georgie Henley
0.3740338274423583
Skandar Keynes
0.43485230451495
William Moseley
0.5881013213584312
The Chronicles of Narnia: Prince Caspian
0.829848285084749
Georgie Henley
0.3740338274423583
Skandar Keynes
0.43485230451495
William Moseley
0.5881013213584312
The Chronicles of Narnia: The Voyage of the Dawn Treader
0.9134254611794095
Georgie Henley
0.3740338274423583
Skandar Keynes
0.43485230451495
Ben Barnes
0.4401314551336654
C.S. Lewis & The Chronicles of Narnia
0.8118548533243488
The Chronicles of Narnia: The Silver Chair
0.9156926263562891
The Chronicles of Narnia: The Lion, the Witch and the Wardrobe /m/chronicles_of_narnia_lion_witch_wardrobe


The Lion, the Witch and the Wardrobe
0.9399286152063473
Arthur Lowe
0.4968304565398925
The Chronicles of Narnia: The Lion, The Witch and The Wardrobe
0.9875352211799856
/m/chronicles_of_narnia_lion_witch_wardrobe 0.9875352211799856
Georgie Henley
0.3740338274423583
Skandar Keynes
0.43485230451495
William Moseley
0.5881013213584312
The Chronicles of Narnia: The Lion, the Witch and the Wardrobe /m/chronicles_of_narnia_lion_witch_wardrobe


Star Wars Ep. V: The Empire Strikes Back to find


Star Wars: Episode V - The Empire Strikes Back
0.8858046971742518
/m/empire_strikes_back 0.8858046971742518
Mark Hamill
0.656010612329752
Harrison Ford
0.6751207995333454
Carrie Fisher
0.6470642812750267
Star Wars Ep. V: The Empire Strikes Back /m/empire_strikes_back


In [76]:
find_movie_df = pd.DataFrame(data=find_movie_list, columns = ['movie', 'url', 'year'])
# select to rows need
find_movie_df = find_movie_df.loc[find_movie_df['url'] != 'to find']

In [77]:
find_movie_df

,movie,url,year
1,Star Wars Ep. VII: The Force Awakens,https://www.rottentomatoes.com/m/star_wars_epi...,2015
4,Star Wars Ep. VIII: The Last Jedi,https://www.rottentomatoes.com/m/star_wars_the...,2017
6,Star Wars Ep. I: The Phantom Menace,https://www.rottentomatoes.com/m/star_wars_epi...,1999
8,Star Wars Ep. IV: A New Hope,https://www.rottentomatoes.com/m/star_wars,1977
9,Pirates of the Caribbean: Dead Manâs Chest,https://www.rottentomatoes.com/m/pirates_of_th...,2006
12,Star Wars Ep. III: Revenge of the Sith,https://www.rottentomatoes.com/m/star_wars_epi...,2005
15,Star Wars Ep. II: Attack of the Clones,https://www.rottentomatoes.com/m/star_wars_epi...,2002
17,Star Wars Ep. VI: Return of the Jedi,https://www.rottentomatoes.com/m/star_wars_epi...,1983
18,"The Chronicles of Narnia: The Lion, the Witch ...",https://www.rottentomatoes.com/m/chronicles_of...,2005
19,"The Chronicles of Narnia: The Lion, the Witch ...",https://www.rottentomatoes.com/m/chronicles_of...,2005


In [69]:
# get movie dataframe wait to insert mongo db
def get_update_movie_list(find_df):
    df = find_df
    data_list = []
    for index, row in df.iterrows():
        data = get_content(row['url'])
        print(row['url'])
        data_list.append(data)
        
    new_df = pd.DataFrame(data=data_list, 
                          columns=['critic(%)', 'critic_rate', 'critic_count', 'critic_fresh', 'critic_rot', 'user(%)', 'user_rate', 'user_number'])
    
    return find_df.reset_index().join(new_df)

In [78]:
ready_df = get_update_movie_list(find_movie_df)
ready_df

https://www.rottentomatoes.com/m/star_wars_episode_vii_the_force_awakens
https://www.rottentomatoes.com/m/star_wars_the_last_jedi
https://www.rottentomatoes.com/m/star_wars_episode_i_the_phantom_menace
https://www.rottentomatoes.com/m/star_wars
https://www.rottentomatoes.com/m/pirates_of_the_caribbean_dead_mans_chest
https://www.rottentomatoes.com/m/star_wars_episode_iii_revenge_of_the_sith
https://www.rottentomatoes.com/m/star_wars_episode_ii_attack_of_the_clones
https://www.rottentomatoes.com/m/star_wars_episode_vi_return_of_the_jedi
https://www.rottentomatoes.com/m/chronicles_of_narnia_lion_witch_wardrobe
https://www.rottentomatoes.com/m/chronicles_of_narnia_lion_witch_wardrobe
https://www.rottentomatoes.com/m/empire_strikes_back


,index,movie,url,year,critic(%),critic_rate,critic_count,critic_fresh,critic_rot,user(%),user_rate,user_number
0,1,Star Wars Ep. VII: The Force Awakens,https://www.rottentomatoes.com/m/star_wars_epi...,2015,93,8.2/10,404,375,29,87,4.3/5,"229,322"
1,4,Star Wars Ep. VIII: The Last Jedi,https://www.rottentomatoes.com/m/star_wars_the...,2017,91,8.1/10,419,381,38,45,2.8/5,"204,091"
2,6,Star Wars Ep. I: The Phantom Menace,https://www.rottentomatoes.com/m/star_wars_epi...,1999,54,6/10,220,119,101,59,3.3/5,"1,206,541"
3,8,Star Wars Ep. IV: A New Hope,https://www.rottentomatoes.com/m/star_wars,1977,93,8.7/10,114,106,8,96,4.1/5,"859,516"
4,9,Pirates of the Caribbean: Dead Manâs Chest,https://www.rottentomatoes.com/m/pirates_of_th...,2006,53,6/10,221,118,103,72,3.9/5,"1,847,533"
5,12,Star Wars Ep. III: Revenge of the Sith,https://www.rottentomatoes.com/m/star_wars_epi...,2005,79,7.3/10,291,231,60,65,3.1/5,"33,682,429"
6,15,Star Wars Ep. II: Attack of the Clones,https://www.rottentomatoes.com/m/star_wars_epi...,2002,66,6.6/10,248,163,85,56,3.3/5,"851,470"
7,17,Star Wars Ep. VI: Return of the Jedi,https://www.rottentomatoes.com/m/star_wars_epi...,1983,81,7.2/10,88,71,17,94,4.1/5,"1,010,844"
8,18,"The Chronicles of Narnia: The Lion, the Witch ...",https://www.rottentomatoes.com/m/chronicles_of...,2005,76,6.9/10,214,162,52,61,3.1/5,"34,105,040"
9,19,"The Chronicles of Narnia: The Lion, the Witch ...",https://www.rottentomatoes.com/m/chronicles_of...,2005,76,6.9/10,214,162,52,61,3.1/5,"34,105,040"


In [79]:
# update and save data in mongo db
for index, row in ready_df.iterrows():
    col.find_one_and_update(
        {
            'name': str(row['movie']),
            'year': int(row['year'])
        },
        {'$set':{
            'url': row['url'],
            'critic(%)': row['critic(%)'],
            'critic_rate': row['critic_rate'],
            'critic_count': row['critic_count'],
            'critic_fresh': row['critic_fresh'],
            'critic_rot': row['critic_rot'],
            'user(%)': row['user(%)'],
            'user_rate': row['user_rate'],
            'user_number': row['user_number']
        }}
    )

In [82]:
# get latest movie pending list and do third round search
goal_list = get_latest_pending_list()
goal_list

,index,name,url,year
0,6,The Avengers,https://www.rottentomatoes.com/m/the_avengers,2012
1,59,It,https://www.rottentomatoes.com/m/it,2017
